# Example 4: Weighted Average
Firstly, let's import some modules that we will depend on in the next examples.

In [1]:
import time
import random
import pickle
import threading
import seal
from seal import ChooserEvaluator, \
	Ciphertext, \
	Decryptor, \
	Encryptor, \
	EncryptionParameters, \
	Evaluator, \
	IntegerEncoder, \
	FractionalEncoder, \
	KeyGenerator, \
	MemoryPoolHandle, \
	Plaintext, \
	SEALContext, \
	EvaluationKeys, \
	GaloisKeys, \
	PolyCRTBuilder, \
	ChooserEncoder, \
	ChooserEvaluator, \
	ChooserPoly

In this example we demonstrate the FractionalEncoder, and use it to compute
 a weighted average of 10 encrypted rational numbers. In this computation we
 perform homomorphic multiplications of ciphertexts by plaintexts, which is
 much faster than regular multiplications of ciphertexts by ciphertexts.
 Moreover, such `plain multiplications` never increase the ciphertext size,
 which is why we have no need for evaluation keys in this example.
 We start by creating encryption parameters, setting up the SEALContext, keys,
 and other relevant objects. Since our computation has multiplicative depth of
 only two, it suffices to use a small poly_modulus.

In [2]:
def print_parameters(context):
    print("/ Encryption parameters:")
    print("| poly_modulus: " + context.poly_modulus().to_string())
    print("| coeff_modulus_size: " + (str)(context.total_coeff_modulus().significant_bit_count()) + " bits")
    print("| plain_modulus: " + (str)(context.plain_modulus().value()))
    print("| noise_standard_deviation: " + (str)(context.noise_standard_deviation()))

parms = EncryptionParameters()
parms.set_poly_modulus("1x^2048 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(2048))
parms.set_plain_modulus(1 << 8)
context = SEALContext(parms)
print_parameters(context)
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

/ Encryption parameters:
| poly_modulus: 1x^2048 + 1
| coeff_modulus_size: 56 bits
| plain_modulus: 256
| noise_standard_deviation: 3.19


We also set up an Encryptor, Evaluator, and Decryptor here.

In [3]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

Create a vector of 10 rational numbers (as doubles).

In [4]:
rational_numbers = [3.1, 4.159, 2.65, 3.5897, 9.3, 2.3, 8.46, 2.64, 3.383, 2.795]

Create a vector of weights.

In [5]:
coefficients = [0.1, 0.05, 0.05, 0.2, 0.05, 0.3, 0.1, 0.025, 0.075, 0.05]

We need a FractionalEncoder to encode the rational numbers into plaintext
polynomials. In this case we decide to reserve 64 coefficients of the
polynomial for the integral part (low-degree terms) and expand the fractional
part to 32 digits of precision (in base 3) (high-degree terms). These numbers
can be changed according to the precision that is needed; note that these
choices leave a lot of unused space in the 2048-coefficient polynomials.

In [6]:
encoder = FractionalEncoder(context.plain_modulus(), context.poly_modulus(), 64, 32, 3)

We create a vector of ciphertexts for encrypting the rational numbers.

We create our Ciphertext objects into the vector by passing the
encryption parameters as an argument to the constructor. This ensures
that enough memory is allocated for a size 2 ciphertext. In this example
our ciphertexts never grow in size (plain multiplication does not cause
ciphertext growth), so we can expect the ciphertexts to remain in the same
location in memory throughout the computation. In more complicated examples
one might want to call a constructor that reserves enough memory for the
ciphertext to grow to a specified size to avoid costly memory moves when
multiplications and relinearizations are performed.

In [7]:
encrypted_rationals = []
rational_numbers_string = "Encoding and encrypting: "
for i in range(10):
    encrypted_rationals.append(Ciphertext(parms))
    encryptor.encrypt(encoder.encode(rational_numbers[i]), encrypted_rationals[i])
    rational_numbers_string += (str)(rational_numbers[i])[:6]
    if i < 9: rational_numbers_string += ", "
print(rational_numbers_string)

Encoding and encrypting: 3.1, 4.159, 2.65, 3.5897, 9.3, 2.3, 8.46, 2.64, 3.383, 2.795


Next we encode the coefficients. There is no reason to encrypt these since they
are not private data.

In [8]:
encoded_coefficients = []
encoded_coefficients_string = "Encoding plaintext coefficients: "
for i in range(10):
    encoded_coefficients.append(encoder.encode(coefficients[i]))
    encoded_coefficients_string += (str)(coefficients[i])[:6]
    if i < 9: encoded_coefficients_string += ", "
print(encoded_coefficients_string)

Encoding plaintext coefficients: 0.1, 0.05, 0.05, 0.2, 0.05, 0.3, 0.1, 0.025, 0.075, 0.05


We also need to encode 0.1. Multiplication by this plaintext will have the
effect of dividing by 10. Note that in SEAL it is impossible to divide
ciphertext by another ciphertext, but in this way division by a plaintext is
possible.

In [9]:
div_by_ten = encoder.encode(0.1)

Now compute each multiplication.

Note how we use plain multiplication instead of usual multiplication. The
result overwrites the first argument in the function call.

In [10]:
print("Computing products: ")
for i in range(10):
    evaluator.multiply_plain(encrypted_rationals[i], encoded_coefficients[i])
print("Done")

Computing products: 
Done


To obtain the linear sum we need to still compute the sum of the ciphertexts
in encrypted_rationals. There is an easy way to add together a vector of
Ciphertexts.

In [11]:
encrypted_result = Ciphertext()
print("Adding up all 10 ciphertexts: ")
evaluator.add_many(encrypted_rationals, encrypted_result)
print("Done")

Adding up all 10 ciphertexts: 
Done


Perform division by 10 by plain multiplication with div_by_ten.

In [12]:
print("Dividing by 10: ")
evaluator.multiply_plain(encrypted_result, div_by_ten)
print("Done")

Dividing by 10: 
Done


How much noise budget do we have left?

In [13]:
print("Noise budget in result: " + (str)(decryptor.invariant_noise_budget(encrypted_result)) + " bits")

Noise budget in result: 31 bits


Decrypt, decode, and print result.

In [14]:
plain_result = Plaintext()
print("Decrypting result: ")
decryptor.decrypt(encrypted_result, plain_result)
print("Done")
result = encoder.decode(plain_result)
print("Weighted average: " + (str)(result)[:8])

Decrypting result: 
Done
Weighted average: 0.382886
